In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from vtl.Readfile import Readfile
from vtl.Photo import Photometry
from spec_id import Highest_likelihood_model_galaxy, Galaxy_ids, Analyze_Stack_avgage_cont_feat_gal_age_correct
import pandas as pd
from glob import glob
from astropy.io import fits, ascii
from astropy.table import Table
from scipy.interpolate import interp1d
import os
import cPickle
import seaborn as sea
sea.set(style='white')
sea.set(style='ticks')
sea.set_style({"xtick.direction": "in","ytick.direction": "in"})
colmap = sea.cubehelix_palette(12, start=2, rot=.2, dark=0, light=1.1, as_cmap=True)

In [3]:
gids=Galaxy_ids('masslist_mar22.dat')
metal=np.arange(0.002,0.031,0.001)
age=np.arange(.5,6.1,.1)

tau=[0,8.0, 8.3, 8.48, 8.6, 8.7, 8.78, 8.85, 8.9, 8.95, 9.0, 9.04, 9.08, 9.11, 9.15, 9.18, 9.2, 9.23, 9.26, 9.28,
     9.3, 9.32, 9.34, 9.36, 9.38, 9.4, 9.41, 9.43, 9.45, 9.46, 9.48]

In [4]:
def Scale_model(mwv,mfl,swv,sfl,ser,rshift):
    imfl = interp1d(mwv * (1+rshift),mfl)(swv)
    C = np.sum(((sfl * imfl) / ser ** 2)) / np.sum((imfl ** 2 / ser ** 2))
    return C

In [7]:
goods_s=Table.read('../../../Clear_data/goodss_mosaic/goodss_3dhst.v4.3.cat',format='ascii')
goods_n=Table.read('../../../Clear_data/goodsn_mosaic/goodsn_3dhst.v4.3.cat',format='ascii')

In [9]:
cnames = list(goods_s.columns)

# for i in range(len(cnames)):
#     if cnames[i][0:2] == 'f_':
#         print cnames[i]
filt_s = ['f_F606Wcand','f_F606W','f_R','f_Rc','f_F775W','f_I','f_F814Wcand','f_F850LP','f_F850LPcand','f_F098M','f_F105W',
          'f_F125W','f_J','f_tenisJ','f_F140W','f_H','f_tenisK','f_Ks','f_IA624','f_IA651','f_IA679','f_IA709','f_IA738',
          'f_IA767','f_IA797','f_IA827','f_IA856']
e_filt_s = ['e_F606Wcand','e_F606W','e_R','e_Rc','e_F775W','e_I','e_F814Wcand','e_F850LP','e_F850LPcand','e_F098M','e_F105W',
          'e_F125W','e_J','e_tenisJ','e_F140W','e_H','e_tenisK','e_Ks','e_IA624','e_IA651','e_IA679','e_IA709','e_IA738',
            'e_IA767','e_IA797','e_IA827','e_IA856']

filt_n = ['f_F606W','f_R','f_Rs','f_I','f_F775W','f_Z','f_F850LP','f_F105W','f_F125W','f_J','f_F140W','f_H','f_Ks']
e_filt_n = ['e_F606W','e_R','e_Rs','e_I','e_F775W','e_Z','e_F850LP','e_F105W','e_F125W','e_J','e_F140W','e_H','e_Ks']

In [10]:
filters=['f_F606Wcand','f_F606W','f_R','f_Rc','f_F775W','f_I','f_F814Wcand','f_F850LP','f_F850LPcand','f_F098M','f_F105W',
          'f_F125W','f_J','f_tenisJ','f_F140W','f_H','f_tenisK','f_Ks','f_IA624','f_IA651','f_IA679','f_IA709','f_IA738',
          'f_IA767','f_IA797','f_IA827','f_IA856','f_F606W','f_R','f_Rs','f_I','f_F775W','f_Z','f_F850LP','f_F105W',
          'f_F125W','f_J','f_F140W','f_H','f_Ks']

f_errors=['e_F606Wcand','e_F606W','e_R','e_Rc','e_F775W','e_I','e_F814Wcand','e_F850LP','e_F850LPcand','e_F098M','e_F105W',
          'e_F125W','e_J','e_tenisJ','e_F140W','e_H','e_tenisK','e_Ks','e_IA624','e_IA651','e_IA679','e_IA709','e_IA738',
          'e_IA767','e_IA797','e_IA827','e_IA856','e_F606W','e_R','e_Rs','e_I','e_F775W','e_Z','e_F850LP','e_F105W',
          'e_F125W','e_J','e_F140W','e_H','e_Ks']

In [15]:
filt_names=['s_F606Wcand','s_F606W','s_R','s_Rc','s_F775W','s_I','s_F814Wcand','s_F850LP','s_F850LPcand','s_F098M','s_F105W',
          's_F125W','s_J','s_tenisJ','s_F140W','s_H','s_tenisK','s_Ks','s_IA624','s_IA651','s_IA679','s_IA709','s_IA738',
          's_IA767','s_IA797','s_IA827','s_IA856','n_F606W','n_R','n_Rs','n_I','n_F775W','n_Z','n_F850LP','n_F105W',
          'n_F125W','n_J','n_F140W','n_H','n_Ks']
effwv = [5818,5818,6418,6460,7634,8549,8001,9001,9001,9841,10476,12410,12311,12513,13831,16451,21534,21638,
         6229,6497,6780,7072,7358,7679,7964,8245,8563,5818,6487,6788,7939,7634,9077,9001,10476,12410,12499,13831,16307,21541]
fwhm = [1550,1550,943,1103,990,1356,1585,1235,1235,1163,1979,2041,1774,1080,2676,1991,2135,1854,225,240,250,
        237,237,262,266,245,266,1550,817,1047,982,990,970,1235,1979,2041,1108,2676,1912,2073]
phot_db = pd.DataFrame({'eff_wv' : effwv, 'fwhm' : fwhm },index = filt_names)

In [17]:
for i in range(len(gids.ids)):
    f_phot = np.repeat(-99.0,len(filters))
    f_err = np.repeat(-99.0,len(filters))
    gal_ID = int(gids.ids[i][1:])
    if gids.ids[i][0] == 's':
        for ii in range(len(filt_s)):
            f_phot[ii] = goods_s[gal_ID-1][filt_s[ii]]
            f_err[ii] = goods_s[gal_ID-1][e_filt_s[ii]]

    if gids.ids[i][0] == 'n':
        for ii in range(len(filt_n)):
            f_phot[ii + len(filt_s)] = goods_n[gal_ID-1][filt_n[ii]]
            f_err[ii + len(filt_s)] = goods_n[gal_ID-1][e_filt_n[ii]]
    
    phot_add = pd.DataFrame({gids.ids[i] : f_phot, '%s_err' % gids.ids[i] : f_err },index = filt_names)
    phot_db = phot_db.join(phot_add)

In [23]:
phot_db

,eff_wv,fwhm,n14713,n14713_err,n16758,n16758_err,n21156,n21156_err,n21427,n21427_err,...,s44620,s44620_err,s45972,s45972_err,s46066,s46066_err,s47677,s47677_err,s48631,s48631_err
s_F606Wcand,5818,1550,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000
s_F606W,5818,1550,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,-99.000000,-99.000000,1.842122,0.042595,0.508798,0.028743,-99.000000,-99.000000,1.360973,0.075360
s_R,6418,943,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,1.042473,0.022198,2.896026,0.027874,0.691281,0.026387,0.820523,0.024696,2.152800,0.051871
s_Rc,6460,1103,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,0.997781,0.056056,2.990469,0.069154,0.773933,0.051897,0.816439,0.053637,2.075598,0.096157
s_F775W,7634,990,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,-99.000000,-99.000000,6.765190,0.072645,2.371791,0.048622,-99.000000,-99.000000,4.521061,0.129567
s_I,8549,1356,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,4.485460,0.300378,13.561190,0.423599,4.134985,0.293417,3.297968,0.296045,9.430043,0.467359
s_F814Wcand,8001,1585,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,2.188003,0.112802,8.995494,0.064019,3.079942,0.038375,-99.000000,-99.000000,6.211490,0.124296
s_F850LP,9001,1235,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,-99.000000,-99.000000,15.831162,0.095238,5.559113,0.067928,-99.000000,-99.000000,10.484480,0.168063
s_F850LPcand,9001,1235,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,-99.000000,-99.000000,15.894367,0.313434,5.517749,0.138862,-99.000000,-99.000000,-99.000000,-99.000000
s_F098M,9841,1163,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,-99.000000,-99.000000,22.564123,0.086817,9.745344,0.054790,-99.000000,-99.000000,15.307628,0.135199


In [19]:
phot_db.to_pickle('phot_db.pkl')

In [6]:
phot_db = pd.read_pickle('phot_db.pkl')

In [8]:
def Plot_with_phot(galaxy,redshift, save_plot=False,plot_name=''):

    wv,fl,er = np.load('spec_stacks_jan24/%s_stack.npy' % galaxy)
    wv,fl,er =np.array([wv[fl >1E-20],fl[fl >1E-20],er[fl >1E-20]])
    IDw = [U for U in range(len(wv)) if 8000 <= wv[U] <= 11100]
        
    Pr,bfa,bfm=Analyze_Stack_avgage_cont_feat_gal_age_correct('chidat/%s_apr6_galfit_cont_chidata.fits' % galaxy,
            'chidat/%s_apr6_galfit_feat_chidata.fits' % galaxy,redshift,np.array(tau),metal,age)
    bfm,bfa,bft=Highest_likelihood_model_galaxy(galaxy,redshift,bfm,bfa,tau)
    mwv,mfl=np.load('../../../fsps_models_for_fit/fsps_spec/m%s_a%s_t%s_spec.npy' % (bfm,bfa,bft))
    
    Clam = 3E18 / phot_db['eff_wv'] **2 * 10**((-1.1)/2.5-29)
    Cm = Scale_model(mwv,mfl,wv,fl,er,redshift)
    
    filters = [240,201,197,198,284]
    effwv = np.repeat(-99.,len(filters))
    phot = np.repeat(-99.,len(filters))
    photer = np.repeat(-99.,len(filters))
    fwhm =np.repeat(-99.,len(filters))
    
    gal_phot = Photometry(wv,fl,er,240)
    
    for i in range(len(filters)):
        gal_phot.Get_Sensitivity(filters[i])
        if wv[0] <= gal_phot.sens_wv[0] and gal_phot.sens_wv[-1] <= wv[-1]:
            gal_phot.Photo()
            gal_phot.FWHM()
            effwv[i],phot[i],photer[i] = [gal_phot.eff_wv,gal_phot.photo,gal_phot.photo_er]
            fwhm[i] = gal_phot.fwhm/2
    if max(phot_db[galaxy]*1E18 * Clam) > max(fl[IDw]*1E18):
        ymax = max(phot_db[galaxy]*1E18 * Clam)
    else:
        ymax = max(fl[IDw]*1E18)
    
    gs=gridspec.GridSpec(2,1)

    plt.figure(figsize=[15,8])
    plt.subplot(gs[0])
    plt.plot(mwv * (1 + redshift),mfl*Cm *1E18, color='k', alpha = .3)
    plt.plot(wv[IDw],fl[IDw]*1E18,color = 'k')
    plt.errorbar(phot_db['eff_wv'],phot_db[galaxy] * Clam *1E18 ,phot_db['%s_err' % galaxy] * Clam *1E18,
                 phot_db['fwhm']/2 ,color='#1D659D',fmt='D',zorder=5)
    plt.errorbar(effwv,phot*1E18,photer*1E18,fwhm,color='#c90316',fmt ='s',zorder=5)
    # plt.xticks
    plt.ylim(0,ymax*1.1)
    plt.xlim(5800,22000)
    plt.minorticks_on()
    plt.ylabel('F$_\lambda$ (10$^{-18}$ erg/s/cm$^2$/$\AA$)',size=20)
    plt.tick_params(axis='both', which='major', labelsize=15)
    plt.title(galaxy,size=20)
    
    plt.subplot(gs[1])
    plt.plot(mwv * (1 + redshift),mfl*Cm*1E18, color='k', alpha = .3)
    plt.plot(wv[IDw],fl[IDw]*1E18,color = 'k')
    plt.errorbar(phot_db['eff_wv'],phot_db[galaxy] * Clam *1E18 ,phot_db['%s_err' % galaxy] * Clam *1E18,
                 phot_db['fwhm']/2 ,color='#1D659D',fmt='D',zorder=5)
    plt.errorbar(effwv,phot*1E18,photer*1E18,fwhm,color='#c90316',fmt ='s',zorder=5)
    plt.ylim(0,ymax*1.1)
    plt.xlim(7900,11300)
    plt.minorticks_on()
    plt.ylabel('F$_\lambda$ (10$^{-18}$ erg/s/cm$^2$/$\AA$)',size=20)
    plt.tick_params(axis='both', which='major', labelsize=15)
    plt.xlabel('Wavelength ($\AA$)',size=20)
    if save_plot == True:
        plt.savefig(plot_name)
    else:
        plt.show()
    plt.close()

In [11]:
%matplotlib inline

for i in range(len(gids.ids)):
    Plot_with_phot(gids.ids[i],gids.rshift[i],save_plot=True,plot_name='../phot_plots/%s_phot.png' % gids.ids[i])

Best fit model is 3.6 Gyr and 0.03 Z
Best fit model is 5.2 Gyr and 0.021 Z


Best fit model is 3.5 Gyr and 0.006 Z


Best fit model is 4.1 Gyr and 0.03 Z


Best fit model is 4.1 Gyr and 0.004 Z


Best fit model is 3.8 Gyr and 0.015 Z

...


Best fit model is 3.4 Gyr and 0.03 Z


Best fit model is 3.4 Gyr and 0.005 Z


Best fit model is 3.9 Gyr and 0.013 Z


Best fit model is 1.4 Gyr and 0.015 Z


Best fit model is 3.9 Gyr and 0.03 Z

In [25]:
filter_names = ['F850LP','F098M','Subaru-IA827','Subaru-IA856','Subaru-suprime-z']
gewv = [9001,9841,8245,8563,9077]
gfwhm = [1235,1163,245,266,970]
gphot_db = pd.DataFrame({'eff_wv' : gewv, 'fwhm' : gfwhm },index = filter_names)

In [27]:
gphot_db

,eff_wv,fwhm
F850LP,9001,1235
F098M,9841,1163
Subaru-IA827,8245,245
Subaru-IA856,8563,266
Subaru-suprime-z,9077,970


In [28]:
filters = [240,201,197,198,284]

for i in range(len(gids.ids)):
    wv,fl,er = np.load('spec_stacks_jan24/%s_stack.npy' % gids.ids[i])
    wv,fl,er =np.array([wv[fl >1E-20],fl[fl >1E-20],er[fl >1E-20]])
    gphot = np.repeat(-99.,len(filters))
    gphoter = np.repeat(-99.,len(filters))
    gal_phot = Photometry(wv,fl,er,240)    
    for ii in range(len(filters)):
        gal_phot.Get_Sensitivity(filters[ii])
        if wv[0] <= gal_phot.sens_wv[0] and gal_phot.sens_wv[-1] <= wv[-1]:
            gal_phot.Photo()
            effwv,gphot[ii],gphoter[ii] = [gal_phot.eff_wv,gal_phot.photo,gal_phot.photo_er]
    gphot_add = pd.DataFrame({gids.ids[i] : gphot, '%s_err' % gids.ids[i] : gphoter },index = filter_names)
    gphot_db = gphot_db.join(gphot_add)

In [30]:
gphot_db

,eff_wv,fwhm,n14713,n14713_err,n16758,n16758_err,n21156,n21156_err,n21427,n21427_err,...,s44620,s44620_err,s45972,s45972_err,s46066,s46066_err,s47677,s47677_err,s48631,s48631_err
F850LP,9001,1235,-9.900000e+01,-9.900000e+01,1.655282e-18,3.382030e-19,2.003905e-18,2.096867e-19,-9.900000e+01,-9.900000e+01,...,-9.900000e+01,-9.900000e+01,2.056985e-18,2.375407e-19,-9.900000e+01,-9.900000e+01,5.833260e-19,2.239912e-19,-9.900000e+01,-9.900000e+01
F098M,9841,1163,9.682768e-19,2.617330e-19,2.008664e-18,2.152191e-19,2.649684e-18,1.346482e-19,7.994261e-19,2.839103e-19,...,8.767648e-19,1.351198e-19,2.200527e-18,1.506057e-19,-9.900000e+01,-9.900000e+01,6.405570e-19,1.405135e-19,1.968022e-18,2.797842e-19
Subaru-IA827,8245,245,7.947088e-19,8.292351e-19,1.447455e-18,6.666587e-19,1.407453e-18,4.010380e-19,-9.900000e+01,-9.900000e+01,...,5.026337e-19,4.123109e-19,1.854709e-18,4.594202e-19,5.299989e-19,6.859244e-19,4.885793e-19,4.360372e-19,1.192332e-18,8.374406e-19
Subaru-IA856,8563,266,-9.900000e+01,-9.900000e+01,1.125222e-18,4.227022e-19,1.457018e-18,2.742420e-19,-9.900000e+01,-9.900000e+01,...,-9.900000e+01,-9.900000e+01,1.846765e-18,3.065970e-19,-9.900000e+01,-9.900000e+01,5.145593e-19,2.929361e-19,-9.900000e+01,-9.900000e+01
Subaru-suprime-z,9077,970,7.824379e-19,3.797168e-19,1.737623e-18,3.102450e-19,2.044850e-18,1.928585e-19,-9.900000e+01,-9.900000e+01,...,7.012145e-19,1.985662e-19,2.024572e-18,2.186666e-19,6.034993e-19,3.089458e-19,5.861945e-19,2.054020e-19,1.889125e-18,4.041955e-19


In [31]:
gphot_db.to_pickle('gphot_db.pkl')